In [2]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [3]:
data_path = "C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\test_data.csv"
factor_path = "C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\Data_Excursus2\\data2.csv"

In [4]:
def prepare_data(data_path, factor_path):
    data = pd.read_csv(data_path)
    factors = pd.read_csv(factor_path)
    factors.drop("Unnamed: 0", axis=1, inplace=True)
    factors["Date"] = pd.to_datetime(factors["Date"])
    factors["month_year"] = pd.to_datetime(factors["Date"]).dt.to_period("M")
    data['month_year'] = pd.to_datetime(data['Date']).dt.to_period('M')
    df = data.merge(factors, on="month_year", how="left")
    df.drop("Date_y", axis=1, inplace=True)
    df = df.rename({"Date_x" : "Date"}, axis=1)
    return df

In [6]:
df = prepare_data(data_path, factor_path)

In [4]:
data = pd.read_csv(data_path)
data['month_year'] = pd.to_datetime(data['Date']).dt.to_period('M')

In [7]:
adidas = df.loc[df["Id"] == "866013"]

In [8]:
adidas

,Id,Date,RET.USD,LMV.USD,BETA,EOJ_Marketcap,BM,month_year,RMRF,SMB,HML,WML
37804,866013,1998-12-31,-0.68,4962.06,0.737476,7901.55,0.126818,1998-12,2.508973,-0.919234,1.783595,2.738842
38337,866013,1999-01-29,-23.45,4928.27,0.732626,7901.55,0.126818,1999-01,2.147219,-4.477452,-2.955716,14.725757
38869,866013,1999-02-26,15.08,3772.56,0.639363,7901.55,0.126818,1999-02,-6.237935,6.028959,1.597353,1.515690
39400,866013,1999-03-31,-6.79,4341.47,0.465504,7901.55,0.126818,1999-03,-3.893391,0.127169,0.942076,-2.676169
39930,866013,1999-04-30,10.50,4046.69,0.513188,7901.55,0.126818,1999-04,4.597983,-3.934642,2.949789,-7.332588
...,...,...,...,...,...,...,...,...,...,...,...,...
185432,866013,2017-08-31,-1.24,47537.94,0.970016,40262.33,0.195497,2017-08,0.772069,0.414603,-0.449214,-0.966824
185976,866013,2017-09-29,0.85,46950.08,0.959901,40262.33,0.195497,2017-09,5.057636,-0.695253,0.426054,1.107276
186519,866013,2017-10-31,-1.61,47347.41,0.956255,40262.33,0.195497,2017-10,1.301131,-2.440036,0.132961,1.272750
187063,866013,2017-11-30,-5.98,46585.36,0.948163,40262.33,0.195497,2017-11,1.487124,0.233265,1.735625,-1.285797


In [9]:
def lin_reg(df):
    lr = LinearRegression()
    X = pd.DataFrame(df["RMRF"])
    y = df["RET.USD"]
    #X["Intercept"] = pd.Series([1 for x in range(len(df.index))])
    #X = X[["Intercept", "RMRF"]]
    mod = lr.fit(X, y)
    return mod

In [10]:
mod = lin_reg(adidas)

In [11]:
mod.coef_

array([0.85015476])

In [12]:
def time_beta(df, start, end):
    filtered_df = df.loc[(df["month_year"] >= start) & (data["month_year"] <= end)]
    mod = lin_reg(filtered_df)
    return mod

In [13]:
mod = time_beta(adidas, "2005-07", "2010-06")

In [14]:
mod.coef_

array([1.20510958])

In [15]:
july_1990 = data.loc[data["month_year"] == "1990-07"]

In [29]:
def fama_macbeth(df, factors):
    X = df[factors]
    y = df["RET.USD"]
    X2 = sm.add_constant(X)
    ols  = sm.OLS(y, X2)
    mod = ols.fit()
    return mod


In [35]:
mod = fama_macbeth(july_1990, "BETA")

C:\Users\Marvin\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [36]:
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                RET.USD   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                   0.01421
Date:                Wed, 28 Apr 2021   Prob (F-statistic):              0.905
Time:                        18:15:22   Log-Likelihood:                -535.02
No. Observations:                 152   AIC:                             1074.
Df Residuals:                     150   BIC:                             1080.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.0934      1.695      5.954      0.0

In [94]:
range(1, len(factors))

range(1, 2)

In [92]:
factors = ["BETA", "LMV.USD"]

In [105]:
def cs_reg_estimates(data, factors):
    columns = factors.copy()
    columns.insert(0, "ym")
    columns.insert(1, "Intercept")
    columns.append("no.obs")
    cs_reg_estimates = pd.DataFrame(columns=columns)
    temp2 = []
    for date in data["month_year"].unique():
        temp = data.loc[data["month_year"] == date]
        mod = fama_macbeth(temp, factors)
        temp2 = [mod.params[i] for i in range(1, len(factors)+1)]
        temp2.insert(0, date)
        temp2.insert(1, mod.params[0])
        temp2.append(temp.shape[0])
        cs_reg_estimates.loc[len(cs_reg_estimates)] = temp2
    
    return cs_reg_estimates

In [106]:
test = cs_reg_estimates(data, factors)

In [107]:
test

,ym,Intercept,BETA,LMV.USD,no.obs
0,1990-07,9.965130,0.769885,-0.000179,152
1,1990-08,-5.499742,-7.297133,-0.000055,154
2,1990-09,-3.715820,-10.042430,-0.000223,155
3,1990-10,1.207534,11.136227,-0.000125,156
4,1990-11,1.559882,-1.382430,0.000163,158
...,...,...,...,...,...
325,2017-08,2.158044,-1.512703,0.000001,547
326,2017-09,3.417220,0.629181,0.000020,544
327,2017-10,0.404356,-1.003274,0.000034,545
328,2017-11,4.387501,-0.654321,-0.000040,546


In [24]:
np.mean(cs_reg_estimates["gamma_zero"])

0.7986017220171844

In [25]:
stats.ttest_1samp(cs_reg_estimates["gamma_zero"], 0)

Ttest_1sampResult(statistic=3.819043982517279, pvalue=0.00016010667234259759)

In [26]:
stats.ttest_1samp(cs_reg_estimates["gamma"], 0)

Ttest_1sampResult(statistic=-0.04037954748356142, pvalue=0.9678150282006315)

In [82]:
data

,Id,Date,RET.USD,LMV.USD,BETA,EOJ_Marketcap,BM,month_year
0,702300,1990-07-31,10.85,1625.23,1.046458,1625.23,0.214170,1990-07
1,702699,1990-07-31,9.53,947.25,1.109317,947.25,0.826878,1990-07
2,729108,1990-07-31,2.57,2474.12,0.550301,2474.12,0.498434,1990-07
3,729175,1990-07-31,-4.40,185.31,1.295822,185.31,0.213723,1990-07
4,729304,1990-07-31,6.46,222.19,0.694801,222.19,0.459050,1990-07
...,...,...,...,...,...,...,...,...
187728,952285,2017-12-29,-0.99,174.34,1.024999,157.00,0.746940,2017-12
187729,952299,2017-12-29,-1.89,968.19,0.309518,780.82,0.540458,2017-12
187730,982910,2017-12-29,6.19,7.01,-0.264888,7.84,0.662751,2017-12
187731,992562,2017-12-29,-2.62,1267.33,1.141478,1114.49,0.418188,2017-12


In [108]:
factors = ["BETA", "EOJ_Marketcap", "BM"]

In [109]:
test2 = cs_reg_estimates(data, factors)

In [110]:
test2

,ym,Intercept,BETA,EOJ_Marketcap,BM,no.obs
0,1990-07,8.660189,1.256183,-1.823763e-04,1.828944,152
1,1990-08,-5.600306,-7.238388,-7.223664e-05,0.149789,154
2,1990-09,-3.560052,-10.071646,-2.120256e-04,-0.257722,155
3,1990-10,0.693561,11.302091,-9.778200e-05,0.762683,156
4,1990-11,1.955190,-1.398218,1.257374e-04,-0.721721,158
...,...,...,...,...,...,...
325,2017-08,2.074390,-1.491653,9.771145e-07,0.088611,547
326,2017-09,3.355697,0.642882,2.170147e-05,0.063244,544
327,2017-10,0.054046,-0.927532,3.695173e-05,0.380048,545
328,2017-11,0.880806,0.034778,-3.239862e-05,3.796822,546


In [ ]:
data_path = "C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\Data_Excursus3\\"
data3a = pd.read_csv(data_path+"data3a.csv")
data3a.drop("Unnamed: 0", axis=1, inplace=True)
data3a.head()

In [ ]:
data3b = pd.read_csv(data_path+"data3b.csv")
data3b.drop("Unnamed: 0", axis=1, inplace=True)
data3c = pd.read_csv(data_path+"data3c.csv")
data3c.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
def factor_spanning_test(df, spanning_factors, test_factor):
    lr = LinearRegression()
    X = df[spanning_factors]
    y = df[test_factor]
    X2 = sm.add_constant(X)
    ols = sm.OLS(y, X2)
    mod = ols.fit()
    print(mod.summary())
    

In [ ]:
factor_spanning_test(data3a, ["RMRF.local", "SMB", "HML"], "WML")

In [ ]:
data3c["Date"] = pd.to_datetime(data3c["Date"])
data3c["month"] = pd.DatetimeIndex(data3c["Date"]).month
data3c["year"] = pd.DatetimeIndex(data3c["Date"]).year
condlist = [data3c["month"] >= 7, data3c["month"] < 7]
choicelist = [data3c["year"] - 1, data3c["year"] - 2]
data3c["hcyear"] = np.select(condlist, choicelist)

In [ ]:
merged_df = data3c.merge(data3b, left_on=["Id", "hcyear"], right_on=["Id", "YEAR"])

In [ ]:
data_path = "C:\\Users\\Marvin\\OneDrive\\Desktop\\Studium\\SS21\\Seminar\\Data_Excursus4\\data4a.csv"
data4a = pd.read_csv(data_path)
data4a.drop("Unnamed: 0", axis=1, inplace=True)
data4a.head()

In [ ]:
def transform_date(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df["month"] = pd.DatetimeIndex(df["Date"]).month
    df["year"] = pd.DatetimeIndex(df["Date"]).year
    return df

In [ ]:
data4a = transform_date(data4a)
data4a.head()

In [ ]:
def mv_lag(df, lag):  
    condlist = [df["month"] >= 7, df["month"] < 7]
    choicelist = [df["year"], df["year"] - lag]
    df["hcoljune"] = np.select(condlist, choicelist)
    return df

In [ ]:
data4a = mv_lag(data4a, 1)
data4a.head()

In [ ]:
fig, axs = plt.subplots(1, 2)
x1 = data4a.loc[(data4a["month"] == 7) & (data4a["year"] == 2011)]["MV.USD.June"]
x2 = np.log(data4a.loc[(data4a["month"] == 6) & (data4a["year"] == 1991)]["MV.USD.June"])
nbins = 50
axs[0].hist(x1, bins=nbins)
axs[1].hist(x2, bins=nbins)
plt.show()

In [ ]:
sorted_data4a = data4a.sort_values(by=["Date", "LMV.USD"], ascending=[True, False])
sorted_data4a

In [ ]:
for date in sorted_data4a["Date"].unique():
    temp = sorted_data4a.loc[sorted_data4a["Date"] == date]
    pfsize = np.array(temp["MV.USD.June"].cumsum() / temp["MV.USD.June"].sum())
    pfsize = ["Small" if i >= 0.9 else "Big" for i in pfsize]
    sorted_data4a.loc[sorted_data4a["Date"] == date, ["pfsize"]] = pfsize